In [1]:
from taxcalc import *
import pandas as pd
import numpy as np
from bokeh.io import show, output_notebook, output_file, save
from bokeh.models import ColumnDataSource, NumeralTickFormatter, Span, HoverTool, Panel, Tabs
from bokeh.plotting import figure

In [31]:
recs = Records()
pol = Policy()
calc = Calculator(records=recs, policy=pol)

In [32]:
calc.advance_to_year(2020)
calc.calc_all()

# eitc_base = calc.weighted_total('eitc')
# ctc_base = calc.weighted_total('c07220') + calc.weighted_total('c11070')
# eitc_base / 1e9

In [4]:
pol._EITC_c[7]

array([ 540.5 , 3604.35, 5952.74, 6697.35])

In [5]:
ratio_1 = pol._EITC_c[7][1] / pol._EITC_c[7][0]
ratio_2 = pol._EITC_c[7][2] / pol._EITC_c[7][0]
ratio_3 = pol._EITC_c[7][3] / pol._EITC_c[7][0]

In [6]:
eitc = 1679
pol_eitc = Policy()
eitc_ref = {'EITC_c': {2020: [eitc, float(eitc * ratio_1), float(eitc * ratio_2), float(eitc * ratio_3)]},
            'CTC_c': {2020: 0.0},
            'ACTC_c': {2020: 0.0}}
pol_eitc.implement_reform(eitc_ref)

calc_eitc = Calculator(records=recs, policy=pol_eitc)
calc_eitc.advance_to_year(2020)
calc_eitc.calc_all()
(calc.weighted_total('iitax') - calc_eitc.weighted_total('iitax')) / 1e9

CTC_c was redefined in release 1.0.0



0.015336960344482422

In [7]:
ctc = 3460
pol_ctc = Policy()
pol_ref = {'EITC_c': {2020: [0,0,0,0]},
           'CTC_c': {2020: ctc},
           'ACTC_c': {2020: ctc}}
pol_ctc.implement_reform(pol_ref)
calc_ctc = Calculator(records=recs, policy=pol_ctc)
calc_ctc.advance_to_year(2020)
calc_ctc.calc_all()
(calc.weighted_total('iitax') - calc_ctc.weighted_total('iitax')) / 1e9

CTC_c was redefined in release 1.0.0



0.027294808671386717

In [8]:
bins = ['<$0K', '=$0K', '$0-10K', '$10-20K', '$20-30K', '$30-40K', '$40-50K',
       '$50-75K', '$75-100K', '$100-200K', '$200-500K', '$500-1000K',
       '>$1000K', 'ALL']

diff_table_eitc = calc.difference_table(calc_eitc, 'standard_income_bins', 'iitax')
diff_eitc = pd.DataFrame()
diff_eitc['count'] = diff_table_eitc['count'] / 100
diff_eitc['perc_inc'] = diff_table_eitc['perc_inc'] / 100
diff_eitc['perc_cut'] = diff_table_eitc['perc_cut'] / 100
diff_eitc['mean'] = diff_table_eitc['mean']
diff_eitc['pc_aftertaxinc'] = diff_table_eitc['pc_aftertaxinc'] / 100
diff_eitc['bins'] = bins

In [9]:
bins = ['<$0K', '=$0K', '$0-10K', '$10-20K', '$20-30K', '$30-40K', '$40-50K',
       '$50-75K', '$75-100K', '$100-200K', '$200-500K', '$500-1000K',
       '>$1000K', 'ALL']

diff_table_ctc = calc.difference_table(calc_ctc, 'standard_income_bins', 'iitax')
diff_ctc = pd.DataFrame()
diff_ctc['count'] = diff_table_ctc['count'] / 100
diff_ctc['perc_inc'] = diff_table_ctc['perc_inc'] / 100
diff_ctc['perc_cut'] = diff_table_ctc['perc_cut'] / 100
diff_ctc['mean'] = diff_table_ctc['mean']
diff_ctc['pc_aftertaxinc'] = diff_table_ctc['pc_aftertaxinc'] / 100
diff_ctc['bins'] = bins

In [15]:
output_notebook()

src_eitc = ColumnDataSource(diff_eitc)
src_ctc = ColumnDataSource(diff_ctc)

bins_graph = ['$0-10K', '$10-20K', '$20-30K', '$30-40K', '$40-50K',
       '$50-75K', '$75-100K', '$100-200K', '$200-500K', '$500-1000K',
       '>$1000K']

Loading BokehJS ...

In [11]:
def mean_fig(src, title):
    p = figure(x_range=bins_graph, plot_height=300, plot_width=750, tools="", title=title)
    p.vbar(x='bins', top='mean', width=0.5, source=src, color='#ff7f0e', alpha=0.7)

    hline = Span(location=0, dimension='width', line_color='gray')
    p.renderers.extend([hline])

    p.yaxis.formatter = NumeralTickFormatter(format='$0,000')
    p.yaxis.axis_label = 'Mean Tax Change'
    p.xaxis.axis_label = 'Income Bin'
    p.xgrid.grid_line_color = None
    p.yaxis.minor_tick_line_color = None

    hover = HoverTool(tooltips='@mean{$0,000}')
    p.add_tools(hover)
    return p

In [12]:
def pc_change(src, title):
    p = figure(x_range=bins_graph, plot_height=300, plot_width=750, tools="", title=title)
    p.vbar(x='bins', top='pc_aftertaxinc', width=0.5, source=src, color='#ff7f0e', alpha=0.7)

    hline = Span(location=0, dimension='width', line_color='gray')
    p.renderers.extend([hline])

    p.yaxis.formatter = NumeralTickFormatter(format='%0.0')
    p.yaxis.axis_label = 'After-Tax Percent Change'
    p.xaxis.axis_label = 'Income Bin'
    p.xgrid.grid_line_color = None
    p.yaxis.minor_tick_line_color = None

    hover = HoverTool(tooltips='@pc_aftertaxinc{%0.0}')
    p.add_tools(hover)
    return p

In [13]:
mean_eitc = mean_fig(src_eitc, "Effects of Replacing CTC With Expanded EITC")
pc_eitc = pc_change(src_eitc, "Effects of Replacing CTC With Expanded EITC")

eitc1 = Panel(child=mean_eitc, title="Mean Tax Change")
eitc2 = Panel(child=pc_eitc, title="After-Tax Percent Change")
tabs_eitc = Tabs(tabs=[eitc1, eitc2])

output_file('EITC_chart.html')
save(tabs_eitc)
# show(tabs_eitc)

'/Users/petermetz/Tax-Calculator/EITC_chart.html'

In [14]:
mean_ctc = mean_fig(src_ctc, "Effects of Replacing EITC With Expanded CTC")
pc_ctc = pc_change(src_ctc, "Effects of Replacing EITC With Expanded CTC")

ctc1 = Panel(child=mean_ctc, title="Mean Tax Change")
ctc2 = Panel(child=pc_ctc, title="After-Tax Percent Change")
tabs_ctc = Tabs(tabs=[ctc1, ctc2])

output_file('CTC_chart.html')
save(tabs_ctc)
# show(tabs_ctc)

'/Users/petermetz/Tax-Calculator/CTC_chart.html'

In [44]:
var_list = ['expanded_income', 'c00100', 'c11070', 'c07220', 'n24', 'num']
df = calc.dataframe(var_list)
df['ctc_tot'] = df['c11070'] + df['c07220']
df_3kids = df.loc[(df['n24']==3) & (df['ctc_tot'] > 0) & (df['num'] == 1)]
df_3kids.describe().round(1)

,expanded_income,c00100,c11070,c07220,n24,num,ctc_tot
count,1899.0,1899.0,1899.0,1899.0,1899.0,1899.0,1899.0
mean,39230.1,34353.4,2811.9,1648.8,3.0,1.0,4460.7
std,105981.4,97881.1,1282.9,1701.5,0.0,0.0,1729.1
min,-4035479.1,-4045110.6,0.0,0.0,3.0,1.0,6.4
25%,24345.5,22448.9,2007.8,232.3,3.0,1.0,3094.3
50%,34356.8,31644.8,3144.2,1195.3,3.0,1.0,5279.3
75%,47915.4,43680.7,3925.5,2414.2,3.0,1.0,6000.0
max,1560665.3,284903.5,4200.0,6000.0,3.0,1.0,6000.0
